In [2]:
import networkx as nx

import numpy as np
from scipy.sparse import csgraph
import scipy.signal as signal
import scipy.linalg as la
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import math
import time
import os

### Iz teorije grafova

```_Laplace_inDegree(Digrapf G)```
* input: graf G
* output: matrica L, Laplaceova in-degree matrica grafa

In [3]:
def _Laplace_inDegree(G):
  A = nx.adjacency_matrix(G)
  A = np.transpose(A)
  D = np.diag(A @ np.ones(np.size(A,1)).transpose())
  return D-A;

Definiramo pojmove
* Za vrh $i \in \mathcal{V}$ definiramo \textbf{dostižni skup (reachable set)} $R(i) = \{ j \in \mathcal{V} : i \rightsquigarrow j\}$.
* *Doseg* $R$ je najveći dostižni skup ili najveći jednostrano dostižni skup.(Nap.dosega može biti više)
*  *Kabal (cabal)* $B \subset R$ je skup vrhova iz kojih se dostiže cijeli $R$. Ako $R$ sadrži samo jedan vrh, taj vrh nazivamo \textbf{korjen}.
* *Ekskluzivni dio* $H \subset R$ je skup vrhova iz $R$ koji ne "vide" vrhove iz drugih dosega.
* *Zajednički dio* $C \subset R$ je skup svih vrhova iz $R$ koji "vide" vrhove iz drugih dosega.

```get_reaches(Digrapf G)```
* input: graf G
* output: vraća listu koja sadrži sve dosege

In [41]:
def get_reaches(G):
    reaches = []
    visited = set()
    V = set(G.nodes())

    for node in V:
        is_new_reach = True
        reach_set = set(nx.descendants(G, node)).union({node})
        #print("\n node:",node,"\n reach_set",reach_set)
        for reach in reaches:
            #print("reach:",reach)
            if reach_set.issubset(reach):
                #print("Nije novi reach.")
                is_new_reach = False
                break
            if reach.issubset(reach_set):
                #print("Nije novi reach.")
                reach.update(reach_set)
                is_new_reach = False
                break                
        if is_new_reach:
            #print("novi reach:",reach_set)
            reaches.append(reach_set)

    return reaches

```get_cabal(Digrapf G, reach reach)```
* input: graf G, doseg reach
* output: skup svih kabala

In [42]:
def get_cabal(G, reach):
    cabal = set()
    for node in reach:
        reachable_set = set(nx.descendants(G,node)).union({node})
        if reach == reachable_set:
            cabal.add(node)
    return cabal

```get_common_part(Digrapf G, reach reach)```
* input: graf G, doseg reach
* output: skup vrhova koji vide vrhove iz drugih reachova (common part)

In [43]:
def get_common_part(G, reach):
    common_part = set()
    all_reaches = get_reaches(G)

    for node in reach:
        sees_other_reach = False
        for other_reach in all_reaches:
            if other_reach != reach:
                if node in other_reach:
                    sees_other_reach = True
                    break
        if sees_other_reach:
            common_part.add(node)

    return common_part

```get_exclusive_part(Digrapf G, reach reach)```
* input: graf G, doseg reach
* output: skup svih vrhova koji se ne vide iz drugih dosega

In [44]:
def get_exclusive_part(G, reach):
    exclusive_part = set()
    all_reaches = get_reaches(G)

    for node in reach:
        sees_other_reach = False
        for other_reach in all_reaches:
            if other_reach != reach and node in other_reach:
                sees_other_reach = True
                break
        if not sees_other_reach:
            exclusive_part.add(node)

    return exclusive_part

```graph_struct(Digrapf G)```
* input: graf G
* output: listu rječnika
        svaki rječnik pripada jednom dosegu i sadrži
```
dict_ = {   "Ri": reach,
            "Bi": cabal,
            "Ci": common_part,
            "Hi": exclusive_part}
```

```print_graph_struct(graph_structure struc)```
* input: graph_structure = rezultat funkcije ```graph_struct``` tj. lista rječnika kao odmah iznad
* output: ispis strukture grafa $G$

In [45]:
def graph_struct(G):
    struc = []
    reaches = get_reaches(G)
    for i,reach in enumerate(reaches):
        cabal = get_cabal(G,reach)
        comm = get_common_part(G,reach)
        excl = get_exclusive_part(G,reach)
        
        dict_ = {
            "R"+str(i+1): reach,
            "B"+str(i+1): cabal,
            "C"+str(i+1): comm,
            "H"+str(i+1): excl
        }
        struc.append(dict_)
    return struc

def print_graph_struct(struc):
    for i,r in enumerate(struc):
        for key in struc[i].keys():
            print(key, "=",r[key])
        print("")

### Iz teorije upravljanja

```mu_(float alpha, float beta, float gamma)```
* input: $\alpha, \beta, \gamma$
* output: ugnježdena funkcija ```M(s)``` ovisna o parametru $s$:  $$\mu(s)=\frac{is\beta -s^2}{is \alpha + \gamma} $$

Primjer poziva:
```
        M = mu_(alpha,beta,gamma)
        print(M(5))
```

In [46]:
def mu_(alpha, beta, gamma):
    def mu(s):
        return (s*s-1j*s*beta)/(gamma + 1j*alpha*s);
    return mu;

```get_Fi(Digraph G)```
*   za graf $G$, parametre $\alpha, \beta, \gamma$, računa $Schurovu$ dekompoziciju inDegree Laplaceove matrice grafa, vraća ugnježdenu funkciju koja prima komponentu $i$ i dimenziju jezgre Laplaceove matrice grafa.

* pritom se kod računanja koristi `scipy.linalg.solve_triangular` koja poziva `LAPACK` rutinu za trokutasti linearni sustav jedndadžbi

* output:   
    * ```Fi(int i, float alpha, float beta, float gamma)```: ugnježdena funkcija
    * ```int Kernel_dim```: dimenzija jezgre Laplaceove matrice
    * ```int ndim```: broj vrhova u grafu G = dimenzija matrice L
    * ```float L_sigma```: po modulu najveća svojstvena vrijednost

Primjer poziva:

    sys_f, Kernel_dim, ndim, L_sigma = get_Fi_norm(graph[0])
            Fi = sys_f(i, alpha, beta, gamma)
```
sys_f, Kernel_dim, ndim, L_sigma = get_Fi(G)
Fi = sys_f(i,alpha,beta,gamma)
print(Fi(5))
```




In [47]:
def get_Fi_norm(G):
    L = _Laplace_inDegree(G)
    ndim = np.shape(L)[0];
    T,Z,dim_Ker_L = la.schur(L, sort = lambda x: abs(x)<1e-12)
    max_diagonal = np.max(np.abs(np.diag(T)))
    #print("- - - - - Računam Schurovu dekompoziciju")
    V = Z[:,dim_Ker_L:]
    EYE = np.diag(np.ones(ndim - dim_Ker_L))
    
    def FF_(i,alpha,beta,gamma):
        alpha_2 = alpha*alpha
        gamma_2 = gamma*gamma
        M = mu_(alpha,beta,gamma)
        VTei = V.T.conj()[:,i-1]
        Z_Tei = Z.T.conj()[dim_Ker_L:,i-1]
        
        def Fi(s):
            if (s == 0):
                #A = T[dim_Ker_L:,dim_Ker_L:] #uzmemo T_22
                x = la.solve_triangular(T[dim_Ker_L:,dim_Ker_L:],VTei)  #solve_triangular poziva
                #LAPACK rutinu za trokutasti sustav
                nrm =la.norm(V @ x /gamma)
                F = nrm
            else:
                x = la.solve_triangular(T[dim_Ker_L:,dim_Ker_L:] - EYE*M(s),Z_Tei)
                F = la.norm(V @ x)/np.sqrt(s*s*alpha_2 + gamma_2)
            return F;
        return Fi;
    return FF_, dim_Ker_L, ndim, max_diagonal;

### Unos grafova

```import_graphs_from_folder_with_index(string folder_path)```
* input: string koji sadrži put mape iz koje želimo učitati grafove
* output: lista tuppleova, a tupple je oblika:
```
(Digraph G, string opis_grafa, int index)
```
```index```: indeks vrha za koji se  $H_\infty$ norma ne dostiže u nuli

In [57]:
def import_graphs_from_folder_with_index(folder_path):
    GRAPHS = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".gexf"):
            file_path = os.path.join(folder_path, file_name)
            
            #iz imena datoteke izvlačimo opis te index i u kojem H_inf nije dostignuta u nuli
            split_parts = file_path.split("_")
            i = int(split_parts[2].split("=")[1])
            graph_name1 = "_".join(split_parts[3:])
            graph_name = graph_name1.split(".gexf")[0] 
            # učitavanje grafa iz datoteke
            G = nx.read_gexf(file_path)
            tup = (G,graph_name,i)
            # dodaj graf u listu
            GRAPHS.append(tup)

    return GRAPHS

In [49]:
# Example usage
folder_path = "Graphs/"
imported_graphs = import_graphs_from_folder_with_index(folder_path)

```analyze_graph(digraph G)```
* input: Digraf G
* output: (maxi_arg, gamma_0, indexi)
    * `maxi_arg`: indeks vrha za koji se dostiže  $H_\infty$ norma
    * `gamma_0`: prvi $\gamma \in \{0.1,0.2,\dots,0.9,1.0\}$ za koji postoji vrh $i$ u kojem se $H_\infty$ norma ne dostiže u nuli
    * `indexi`: lista indeksa u kojima se  $H_\infty$ ne dostiže u nuli
    
Funkcija `analyze_graph(G)` za dani graf $G$ i za svaki $\gamma \in \{0.1,0.2,\dots,0.9,1.0\}$ računa funkciju transfera $F_i$ za vrh $i$ te pomoću Nelder-Meadovog algoritma ispituje gdje se dostiže $max$ funkcije $||F_i(s)||$. Ako se $max$ ne dostiže u nuli, sprema indeks $i$ u listu `indexi`

In [51]:
def analyze_graph(graph):
    struc = graph_struct(graph[0])
    #print_graph_struct(struc)
    alpha = 1
    beta = 1
    sys_f, Kernel_dim, ndim, L_sigma = get_Fi_norm(graph[0])
    print("Broj dosega Ri = dim(KerL) = ",Kernel_dim)
    
    #priprema za Nelder-Mead
    x0 = 0.1
    left_end = 0.0
    right_end = L_sigma * 10
    bounds = [(left_end, right_end)]
    
    not_in_list = True
    arg_i = -1   
    indexi = []
    maxi_arg = []
    gamma_0 = 0.01
    dict_ = {}
    for k in range(1, 11):
        gamma = k / 10
        norm_values = []
        
        for i in range(ndim):
            Fi = sys_f(i, alpha, beta, gamma)
            res = minimize(lambda s: -Fi(s), x0, bounds=bounds, method='Nelder-Mead')
            s_opt = res.x[0]
            norm = -res.fun
            norm_values.append(norm)
            
            if s_opt == 0.0:
                continue            
            elif not_in_list:
                #print("kontra")
                not_in_list = False
                indexi.append(i)
                gamma_0 = gamma
        arg_stari = arg_i
        maxi = max(norm_values)
        arg_i = norm_values.index(maxi)
        if (arg_stari == -1):
            maxi_arg.append(arg_i)
        elif (arg_i != arg_stari):
            print(arg_i,arg_stari)
            print("promjena arg_max_i")
            maxi_arg.append(arg_i)
            
    if not not_in_list:
        #print("arg_maxF_i = ",maxi_arg)
        #print("[indeks prije,indeks sada] = [", graph[2],indexi,"]")
        dict_["arg_max_i"] = maxi_arg
        #dict_["gamma_0"] = gamma
        dict_["indexi"] = indexi
        #ttt = 0
        
    return (maxi_arg, gamma_0, indexi)

```s_gamma_plot(dictionary item, int j)```
* input: `dict item{'graf', 'opis', 'gamma0','indexi'}`, `int j`
* output: plot grafa $s-\gamma$ gdje je na x-osi vrijednost parametra $\gamma$, a na y-osi točka $s$ u kojoj za dani vrh $i$ funkcija prijenosa $F_i$ ne dostiže $max$ u nuli

parametar $j$ služi za export sličice

In [56]:
def s_gamma_plot(item,j):
    G = item["graf"]
    opis = item["opis"]
    gamma_0 = item["gamma_0"]
    indexi = item["indexi"]
    
    alpha = 1
    beta = 1
    sys_f, Kernel_dim, ndim, L_sigma = get_Fi_norm(G)
    
    #priprema parametara za Nelder-Mead
    x0 = 0.1
    left_end = 0.0
    right_end = L_sigma * 10
    bounds = [(left_end, right_end)]
# tražimo minimalni gamma za koji postoji vrh i t.d. se H_inf norma ne dostiže u nuli, ali on nije minimalan,
# znamo da za gamma_0 takav vrh i postoji vrh, ali gamma_0 nije minimalan,
# za gamma_0-0.1 takav vrh još ne postoji, pa testiramo koji je to minimalni gamma u segmentu [gamma_0-0.1 ,  gamma_0]
    gamma = gamma_0-0.09
    for i in indexi:
        gamma_values = [] # vrijednosti na x-osi
        s_opt_values = [] # vrijednosti na y-osi
        while(gamma <=1.0):
            Fi = sys_f(i, alpha, beta, gamma)
            res = minimize(lambda s: -Fi(s), x0, bounds=bounds, method='Nelder-Mead')
            s_opt = res.x[0]
            gamma_values.append(gamma)
            s_opt_values.append(s_opt)
            gamma = gamma+0.01

# plotanje grafa s-gamma
        plt.figure(figsize=(3,3))
        plt.plot(gamma_values,s_opt_values)
        plt.xlabel('gamma')
        plt.ylabel('s_opt')
        plt.title(opis)
        filename = f"Plots/{j:02d}_i={indexi[0]}_{opis}.png"
        #plt.savefig(filename)
        plt.show()

```plot_graph(dictionary item, node uu, node vv int j)```
* input: 
    `dict item{'graf', 'opis', 'gamma0','indexi'}`, 
    `node uu`: vrh u kojem se dostiže $H_\infty$ norma
    `node vv`: lista vrhova u kojima se  $H_\infty$ norma ne dostiže u nuli
    `int j`: služi za export sličice
* output: plot grafa G, crveno je označen vrh `uu`, a zeleno vrhovi `vv`

In [53]:
def plot_graph(item,uu,vv,j):
    Graph = item["graf"]
    opis = item["opis"]
    indexi = item["indexi"]
    u = str(uu[0])
    v = str(vv[0])
    plt.figure(figsize=(5,5))
    #pos = nx.spring_layout(Graph)
    if not Graph.has_node(u) or not Graph.has_node(v):
        print("One or more nodes are missing in the graph.")
        return
    
    pos = nx.nx_agraph.graphviz_layout(Graph, prog='dot')
    
    nodes = nx.draw_networkx_nodes(Graph, pos, node_size=300, nodelist=[u], node_shape='d',node_color='firebrick')  # Diamond shape
    
    nodes = nx.draw_networkx_nodes(Graph, pos, node_size=300, nodelist=[v], node_shape='p',node_color='forestgreen')  # Square shape
    plt.legend([' - max',' - max nije u 0'])
    other_nodes = []
    pos1 = {}
    for node in Graph.nodes():
        if node != u and node != v:
            other_nodes.append(node)
            pos1[node]=pos[node]
    nx.draw_networkx_nodes(Graph, pos1, node_size=200, nodelist=other_nodes,node_color='royalblue')
    
    labels = {node: node for node in Graph.nodes()}
    nx.draw_networkx_labels(Graph, pos, labels=labels, font_color='white', verticalalignment='center')
    edgclr = (224/255.0,229/255.0,255/255.0)
    nx.draw_networkx_edges(Graph, pos,edge_color=edgclr)
        
    plt.title(opis)
    plt.axis('off')
    
    plt.title(opis)
    filename = f"Graph_plots/{j:02d}_i={indexi[0]}_{opis}.png"
    #plt.savefig(filename)
    plt.show()

In [55]:
j = 1;
graph_collection = []

for G in imported_graphs:
    dict_ = {"graf":G[0],"opis":G[1]}
#ispis bridova u grafu G[0]
    #print(G[0].edges())            
    print("\n",j," --  --  --  --  --  --  --  --  --  --  --")  
    
#analiziraj graf: pronađi vrh u kojem se dostiže H_inf norma te vrh i u kojem se H_inf(Fi) ne dostiže u nuli
    T = analyze_graph(G)            #vraća tupple(maxi_arg, gamma_0, indexi)
    
# nađi sve dosege i pripadne kabale, common_parts i exclusive_parts
    struc = graph_struct(G[0])
    
    print("arg_max_i : ",T[0], "   | max nije u nula:",T[2] )

#isprintaj strukturu grafa G
    #print_graph_struct(struc)
    dict_["arg_max_i"] = T[0]
    dict_["gamma_0"] = T[1]
    dict_["indexi"] = T[2]
    #print(T[0],T[2])
    graph_collection.append(dict_)
    #plot_graph(graph_collection[j-1],T[0],T[2],j)    
    #s_gamma_plot(graph_collection[j-1],j)
    
    #print(graph_collection[j-1])
# ovo je bilo za provjeru ispis prvih 15 grafova
    j = j+1
    if j==15:
        break

[('0', '2'), ('0', '6'), ('1', '0'), ('1', '6'), ('2', '3'), ('2', '5'), ('3', '2'), ('3', '5'), ('4', '0'), ('5', '3'), ('7', '6'), ('8', '1'), ('9', '7')]

 1  --  --  --  --  --  --  --  --  --  --  --
Broj dosega Ri = dim(KerL) =  3
arg_max_i :  [9]    | max nije u nula: [2]
{'graf': <networkx.classes.digraph.DiGraph object at 0x00000246966B3E90>, 'opis': 'scale_free_directed_graph(10)', 'arg_max_i': [9], 'gamma_0': 0.9, 'indexi': [2]}
[('0', '2'), ('1', '0'), ('1', '5'), ('2', '1'), ('2', '0'), ('2', '5'), ('3', '2'), ('3', '1'), ('3', '4'), ('4', '3'), ('5', '1'), ('6', '3'), ('7', '1'), ('8', '7'), ('9', '6')]

 2  --  --  --  --  --  --  --  --  --  --  --
Broj dosega Ri = dim(KerL) =  2
arg_max_i :  [0]    | max nije u nula: [7]
{'graf': <networkx.classes.digraph.DiGraph object at 0x000002469658FC50>, 'opis': 'scale_free_directed_graph(10)', 'arg_max_i': [0], 'gamma_0': 0.9, 'indexi': [7]}
[('1', '0'), ('2', '0'), ('3', '1'), ('4', '3'), ('5', '4'), ('6', '3'), ('7', '6'), ('8

In [ ]:
j = 1;
graph_collection = []
#modified_graphs = 
#for G in imported_graphs:
G = (imported_graphs[0][0].copy(),imported_graphs[0][1],imported_graphs[0][2])
print(G[0].edges())
G[0].add_edge(9,1)
#print(G[0].edges())
dict_ = {"graf":G[0],"opis":G[1]}
#print(G[0].edges())
print("\n",j," --  --  --  --  --  --  --  --  --  --  --")    
T = analyze_graph(G)
print("arg_max_i : ",T[0], "   | max nije u nula:",T[2] )
dict_["arg_max_i"] = T[0]
dict_["gamma_0"] = T[1]
dict_["indexi"] = T[2]
    #print(T[0],T[2])
graph_collection.append(dict_)
plot_graph(graph_collection[j-1],T[0],T[2],j)
time.sleep(8)
#s_gamma_plot(graph_collection[j-1],j)

struc = graph_struct(G[0])
print_graph_struct(struc)
#print(graph_collection[j-1])
j= j+1

#if j==15:
#    break

In [53]:
Veerman = nx.DiGraph()
Veerman.add_edges_from([(1, 2), (1, 6), (6, 7), (7, 6), (3, 7), (3, 4), (4, 5), (5, 3)])

Veerman_reaches = get_reaches(Veerman)
for reach in Veerman_reaches:
    print("\nR =", reach)
    Ve_cabal = get_cabal(Veerman,reach)
    print("B =",Ve_cabal)
    Ve_common = get_common_part(Veerman,reach)
    print("C =",Ve_common)
    Ve_excl = get_exclusive_part(Veerman,reach)
    print("H =",Ve_excl)


R = {1, 2, 6, 7}
B = {1}
C = {6, 7}
H = {1, 2}

R = {3, 4, 5, 6, 7}
B = {3, 4, 5}
C = {6, 7}
H = {3, 4, 5}
